# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [14]:
""" Detecting line segments on single pictures and videos which have been made by a camera in a car. 
    Programm: Udacity - Self Driving Car Engineer
    Project: P1 Finding Lane Lines on the Road
    Author: Daniel Gattringer (gada1982) """

#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import math
import string
import os
import numpy as np
import cv2
%matplotlib inline

""" Do some configuration for the programm """
# Print some debug infos when 1, don't do it when 0
# 0 is the default setting for the final solution
printInfos = 0

# Print raw lines if 1, print extrapolated lines if 0
# 0 is the default setting for the final solution
print_raw = 0

# Should the single pictures in the defined folder be modified? 
# Set to 1 if yes, set to 0 if not
# 1 is the default setting for the final solution
modify_pictures_in_folder = 1
""" End of configuration area """

""" Global variables used to smooth the extrapolated line segments in the videos. 
    The mean value is formed from two frames. 
    Shortcomings: Direct passing for global variables is bad style
                    Don't start with fixed values for x and y. """
avg_r_slope_old = 0.65
avg_r_x_old = 660
avg_r_y_old = 420
avg_l_slope_old = -0.65
avg_l_x_old = 330
avg_l_y_old = 380

def grayscale(img):
    """Applies the Grayscale transform and a first filter
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Apply a first filter
    gray_img_opt = cv2.inRange(gray_img, 196, 255)
    return gray_img_opt
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    Draws extrapolated lines on a single frame.
    One for left and on for right lane line.
    """
    # use the global variable for flattening between frames
    # Shortcomings: Direct passing for global variables is bad style
    global avg_r_slope_old
    global avg_r_x_old
    global avg_r_y_old
    global avg_l_slope_old
    global avg_l_x_old
    global avg_l_y_old 
    
    #Define different arrays and initialize y_min and y_max
    right_slope = []
    right_x = []
    right_y = []
    left_slope = []
    left_x = []
    left_y = []
    y_min = int(img.shape[0] / 1.69)
    y_max = img.shape[0]
    
    #Calculate slope and assign them to left/right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            #identify global minimum y
            if y1<y_min:
                y_min = y1
            #Calculate slope and assign it likewise
            slope = (y2-y1)/(x2-x1)
            if 0.8 > slope > 0.4: # Right lane, negative slope --> Limits to sort out horizontal lines
                right_slope.append(slope)
                right_x.append(x1)
                right_x.append(x2)
                right_y.append(y1)
                right_y.append(y2)
            elif -0.8 < slope < -0.4: # Left lane, positive slope --> Limits to sort out horizontal lines
                left_slope.append(slope)
                left_x.append(x1)
                left_x.append(x2)
                left_y.append(y1)
                left_y.append(y2)

    # Calculate the average position of line segments on the right side
    if float(len(right_slope)) != 0:
        avg_right_slope = sum(right_slope)/float(len(right_slope))
        avg_r_slope_old = (avg_r_slope_old + avg_right_slope) / 2
        # Only for debugging
        #print("R_slope: " + str(avg_right_slope))
    else:
        avg_right_slope = avg_r_slope_old
    if float(len(right_x)) != 0:
        avg_right_x = sum(right_x)/float(len(right_x))
        avg_r_x_old = (avg_r_x_old + avg_right_x) / 2
        # Only for debugging
        #print("R_X: " + str(avg_right_x))
    else:
        avg_right_x = avg_r_x_old
    if float(len(right_y)) != 0:
        avg_right_y = sum(right_y)/float(len(right_y))
        avg_r_y_old = (avg_r_y_old + avg_right_y) / 2
        # Only for debugging
        #print("R_Y: " + str(avg_right_y))
    else:
        avg_right_y = avg_r_y_old
    # Calculate intercept for averaged right lines
    right_intercept = avg_right_y-(avg_right_slope*avg_right_x)
    # Find x1,x2 for the highest point on the right line
    x1_right = (y_min-right_intercept)/avg_right_slope
    x2_right = (y_max-right_intercept)/avg_right_slope
    # Draw final right line
    cv2.line(img, (int(x1_right), y_min), (int(x2_right), y_max), color, thickness)
    
    # Calculate the average position of line segments on the left side
    if float(len(left_slope)) != 0:
        avg_left_slope= sum(left_slope)/float(len(left_slope))
        avg_l_slope_old = (avg_l_slope_old + avg_left_slope) / 2
        # Only for debugging
        #print("L_slope: " + str(avg_left_slope))
    else:
        avg_left_slope = avg_l_slope_old
    if float(len(left_x)) != 0:
        avg_left_x = sum(left_x)/float(len(left_x))
        avg_l_x_old = (avg_l_x_old + avg_left_x) / 2
        # Only for debugging
        #print("L_X: " + str(avg_left_x))
    else:
        avg_left_x = avg_l_x_old
    if float(len(left_y)) != 0:
        avg_left_y = sum(left_y)/float(len(left_y))
        avg_l_y_old = (avg_l_y_old + avg_left_y) / 2
        # Only for debugging
        #print("L_y: " + str(avg_left_y))
    else:
        avg_left_y = avg_l_y_old
    
    # Calculate intercept for averaged left lines
    left_intercept = avg_left_y-(avg_left_slope*avg_left_x)
    # Find x1,x2 for the highest point on the left line
    x1_left = (y_min-left_intercept)/avg_left_slope
    x2_left = (y_max-left_intercept)/avg_left_slope
    # Draw final left line
    cv2.line(img, (int(x1_left), y_min), (int(x2_left), y_max), color, thickness)
    
def draw_lines_raw(img, lines, color=[0, 255, 0], thickness=2):
    """
    Draws raw lines on a single frame.
    This can be used for debugging issues
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
              cv2.line(img, (int(x1), y1), (int(x2), y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, raw_lines):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """  
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    # Splits up here depending if raw data lines or extrapolated lines should be shown
    if raw_lines == 1:
        draw_lines_raw(line_img, lines, color=[255, 0, 0], thickness=2)
    else:
        draw_lines(line_img, lines, color=[255, 0, 0], thickness=10)
        
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

def modify_img(img):
    """
    Input: `img` should be a completely unmodified frame
    Output: 'final_img' is the annotated frame
    """
    # Create a gray-colored image
    gray = grayscale(img)
    if printInfos == 1:
        #printing out some stats and plotting
        print('Converted to gray.')
        plt.imshow(gray, cmap='gray')
    
    # Use a GausssianBlur on the gray image
    kernel_size = 5
    blur_gray = gaussian_blur(gray, kernel_size)

    # Define parameters for Canny and apply
    low_threshold = 100
    high_threshold = 200
    edges = canny(blur_gray, low_threshold, high_threshold)
    if printInfos == 1:
        #printing out some stats and plotting
        print('Canny algorithm.')
        plt.imshow(edges, cmap='gray')
        
    # Apply a region of interest
    imshape = img.shape
    vertices = np.array([[(imshape[1] / 8,imshape[0]),(imshape[1] / 2.18, imshape[0] / 1.69), (imshape[1] / 1.85, imshape[0] / 1.69), (imshape[1] / 1.091,imshape[0])]], dtype=np.int32)
    masked_edges = region_of_interest(edges, vertices)
    if printInfos == 1:
        #printing out some stats and plotting
        print('region of interest.')
        plt.imshow(masked_edges, cmap='gray')
        
    # Define the Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    
    image_lines = hough_lines(masked_edges, rho, theta, threshold, min_line_length, max_line_gap, print_raw)
    final_img = weighted_img(image_lines, img, α=0.8, β=1., λ=0.)
    
    return final_img

def mask_color_hsv(img):
    """
    Applies an image mask in hsv color space to filter out yellow and white elements.
    
    IMPORTANT: Not used at the moment. Perhaps useful for further improvements.
    """
    lower_range_yellow = np.array([20,100,100], dtype=np.uint8)
    upper_range_yellow = np.array([30, 255, 255], dtype=np.uint8)
    lower_range_white = np.array([0,0,100], dtype=np.uint8)
    upper_range_white = np.array([0, 0, 255], dtype=np.uint8)

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask_yellow = cv2.inRange(hsv, lower_range_yellow, upper_range_yellow)
    mask_white = cv2.inRange(hsv, lower_range_white, upper_range_white)

    mask = mask_yellow + mask_white

    binary_img = cv2.bitwise_and(hsv, hsv, mask=mask)
    binary_img = cv2.cvtColor(binary_img, cv2.COLOR_HSV2BGR)
    _, binary_img = cv2.threshold(binary_img, 127, 255, cv2.THRESH_BINARY)

    return binary_img
    
if modify_pictures_in_folder == 1:
    filelist = os.listdir("test_images/")

    for filename in filelist:
        path = 'test_images/' + filename
        if path.find("/.") <= 0 and path.find("_modified.jpg") <= 0: # Don't do it for config-files and still modfied files
            #reading in an image
            image = mpimg.imread(path)
            pathsave = path.replace(".jpg","") + "_modified.jpg"
            #print(pathsave)
            finalimage = modify_img(image)
            #plt.imshow(finalimage)
            mpimg.imsave(pathsave, finalimage)
    

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [15]:
# Import everything needed to edit/save/watch video clips
import moviepy
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [16]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    
    result = modify_img(image)

    return result

Let's try the one with the solid white lane on the right first ...

In [17]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|█████████▉| 221/222 [00:18<00:00, 14.84it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

CPU times: user 3.94 s, sys: 1.49 s, total: 5.43 s
Wall time: 21.2 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [19]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|█████████▉| 681/682 [00:59<00:00, 11.52it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

CPU times: user 12 s, sys: 4.64 s, total: 16.7 s
Wall time: 1min


In [20]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

* My line detection software:
 * The written programm can produce annotated pictures (in the test_images folder) and annoted videos. It is working   quite well for the two mandatory videos and for the addtional challenge video too. Anyway it's not perfect. There are still some improvments for further work.

 * The programm can be configured if there shall be additional debug output, if the test pictures in the specific folder shall be annoted and finally if the raw hough lines shall be shown or the extrapolated lines. To do this, please modify the parameters in the configuration area at the beginning of the code.


* The pipeline itself:
 * Convert the picture (frame) into a gray colored one. At this step a first filter is used bei only showing regions  in the range between 196 and 255.
 * Use a Gausssian-Blur on the gray image with kernel-size 5 (because this has shown the best results)
 * Apply the Canny-algorithm with a low-threshold of 100 and a high-threshold of 200
 * Apply a region of interest to mask out everything what is outside of the actual used driving lane. The size of the region is automatically adjusted to the size of the picture (frame)
 * Do a hough transformation with the following parameters: rho = 1, theta = np.pi/180, threshold = 15,    min_line_length = 20 and max_line_gap = 10
 * Create a weighted image (frame) with the following parameters: α=0.8, β=1. and λ=0.
 * Final image (frame) available
 
* Optimizations that have been implemented
 * First filter directly on the gray-scaled image
 * Minimization of fluttering by averaging between two frames
 * Filtering out of horizontal lines through boundaries of the slopes (between +/-0.8 and +/-0.4)
 * Starting values for x/y-values depending on framesize
 * Region of interest depending on framesize
 
* Shortcomings
 * Annotated lines are still fluttering especially when the road is gray (in different shades)
 
* Further improvements
 * Use transformation into HSV-Color-Space and apply filtering for yellow and white segments. This was working quite well for yellow lines, but not for white (light-gray) ones.
 * Optimize the parameters all over the pipeline
 * Check the solutions with further training/test data.
 * Do not directly use global variables for connecting the data from two frames, because this is not a good programming style (write access form inside a function).
 
Thats all. The project was really fun. I am looking forward to the next ones.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [21]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|██████████| 251/251 [00:45<00:00,  7.04it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

CPU times: user 7.68 s, sys: 3.23 s, total: 10.9 s
Wall time: 50.1 s


In [22]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))